# ***PyCaret Extended Tutorial for Classification, Regression and Clustering***


# *1. Evaluate 3 different types of modelling*
* **Classification:** We will use Titanic challenge dataset
* **Regression:** We will use House prices advanced regression challenge dataset
* **Clustering:** We will use Titanic challenge dataset again




# *2. Installing necessary libraries*

In [ ]:
!pip install pycaret
!pip install pandas_profiling

# *3. Importing neccessary libraries*

In [ ]:
import pandas as pd 
import pandas_profiling as pp

# *4. Read the data*

In [ ]:
#read the data
train_titanic=pd.read_csv('/kaggle/input/titanic/train.csv')
test_titanic=pd.read_csv('/kaggle/input/titanic/test.csv')
train_house=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_house=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
#data dimensions
print('Titanic train:',train_titanic.shape)
print('Titanic test:',test_titanic.shape)
print('House price train:',train_house.shape)
print('House price test:',test_house.shape)

# *5. Classification*

In [ ]:
#pandas profiling report for Titanic train data
pp.ProfileReport(train_titanic)

# ****Warning: PyCaret setup will work for last imported PyCaret library. 
# So, if you import clustering library, you will get an error for classification setup****

In [ ]:
#import pycaret classification library
from pycaret.classification import * #for classification
#classification setup
classification_setup =setup(data = train_titanic, 
             target = 'Survived',
             numeric_imputation = 'mean', #fill missing value with mean for numeric features
             categorical_features = ['Sex','Embarked','Pclass','Ticket','Cabin'], #we know categorical features from pandas profiling report
             ignore_features = ['Name','PassengerId'],
             train_size=0.8, #0.7 as default
             high_cardinality_features=['Cabin'],
             normalize=True,
             normalize_method='minmax',
             handle_unknown_categorical=True,
             unknown_categorical_method='most_frequent',  #fill missing value with most frequent value for categorical features
             remove_outliers=True, #it automatically applies PCA for removing outliers,
             outliers_threshold=0.05, #By default, 0.05 is used which means 0.025 of the values on each side of the distribution’s tail are dropped from training data.
             silent=True,
             profile=True #a data profile for Exploratory Data Analysis will be displayed in an interactive HTML report. It also generates pandas profiling report
     )

In [ ]:
#comparing models
blacklist_models = ['svm','rbfsvm','mlp']

compare_models(
    blacklist=blacklist_models, #blacklisted models won't work.
    fold = 5,
    sort = 'Accuracy', ## competition metric
    turbo = True
)

In [ ]:
#creating model with selected estimator.
xgb=create_model(estimator='xgboost',fold=5)

In [ ]:
#tune the model
tuned_xgb = tune_model('xgboost')

In [ ]:
# ensembling a trained xgboost model
xgb_bagged = ensemble_model(xgb)

In [ ]:
plot_model(xgb, plot = 'boundary')# Decision Boundary

In [ ]:
plot_model(xgb, plot = 'pr')# Precision Recall Curve

In [ ]:
plot_model(xgb, plot = 'vc')# Validation Curve

In [ ]:
plot_model(xgb, plot='confusion_matrix') # Confusion Matrix

In [ ]:
#Evaluating model is a good option. Because you don't need to plot different plots seperately. It provides all of them in the same cell.
evaluate_model(xgb)

In [ ]:
#As you remember, we split 80% of the data for training. The rest of the data can be used for holdout prediction.
xgb_holdout_pred = predict_model(xgb)

In [ ]:
#Or you can use your test data for prediction.
titanic_prediction =  predict_model(xgb, data=test_titanic)
titanic_prediction.head()

In [ ]:
#prepare the submission file
titanic_prediction['Survived'] = round(titanic_prediction['Score']).astype(int)
submission=titanic_prediction[['PassengerId','Survived']]
submission.to_csv('submission.csv',index=False)
submission.head()

# *6. Regression*

In [ ]:
#pandas profiling report for House price train data
pp.ProfileReport(train_house)

In [ ]:
from pycaret.regression import * #for regression

#regression setup
regression_setup =setup(data = train_house, 
             target = 'SalePrice',
             numeric_imputation = 'mean', #fill missing value with mean for numeric features
             categorical_features = ['MSZoning','Exterior1st','Exterior2nd','KitchenQual','Functional','SaleType',
                                     'Street','LotShape','LandContour','LotConfig','LandSlope','Neighborhood',   
                                     'Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',    
                                     'MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond',   
                                     'BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir',   
                                     'Electrical','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive',
                                     'SaleCondition'], #we know categorical features from pandas profiling report
             ignore_features = ['Id'],
             train_size=0.8, #0.7 as default
             normalize=True,
             normalize_method='minmax',
             handle_unknown_categorical=True,
             unknown_categorical_method='most_frequent',  #fill missing value with most frequent value for categorical features
             remove_outliers=True, #it automatically applies PCA for removing outliers,
             outliers_threshold=0.05, #By default, 0.05 is used which means 0.025 of the values on each side of the distribution’s tail are dropped from training data.
             silent=True,
             profile=True #a data profile for Exploratory Data Analysis will be displayed in an interactive HTML report. It also generates pandas profiling report
     )

In [ ]:
bl_models = ['ransac', 'tr', 'rf', 'et', 'ada', 'gbr']

compare_models(
    blacklist = bl_models,
    fold = 5,
    sort = 'MAE', ## competition metric
    turbo = True
)

In [ ]:
#creating model.
lgbm = create_model(
    estimator='lightgbm',
    fold=5
)

In [ ]:
#Evaluating model.
evaluate_model(lgbm)

In [ ]:
# use rest of the training data for holdout prediction.
lgbm_holdout_pred = predict_model(lgbm)

In [ ]:
#prediction with test data.
house_prediction =  predict_model(lgbm, data=test_house)
house_prediction.head()

In [ ]:
#prepare the submission file
house_prediction.rename(columns={'Label':'SalePrice'}, inplace=True)
house_prediction[['Id','SalePrice']].to_csv('submission_house.csv', index=False)

# *7. Clustering*

In [ ]:
from pycaret.clustering import * #for clustering
#clustering setup doesn't support silent True options. So, you need to hit ENTER manually.
#if silent is set True, it means that you approve data types which were inferred from PyCaret.

#clustering setup
clustering_setup =setup(data = train_titanic, 
             numeric_imputation = 'mean', #fill missing value with mean for numeric features
             categorical_features = ['Sex','Embarked','Pclass','Ticket','Cabin'], #we know categorical features from pandas profiling report
             ignore_features = ['Name','PassengerId'],
             high_cardinality_features=['Cabin'],
             normalize=True,
             normalize_method='minmax',
             handle_unknown_categorical=True,
             unknown_categorical_method='most_frequent',  #fill missing value with most frequent value for categorical features
             verbose=False        
    
     )

In [ ]:
#use k-means for clustering. You can check other clustering algorithms from https://pycaret.org/clustering/
kmeans = create_model('kmeans')

In [ ]:
#assigning data to clusters.
kmeans_df = assign_model(kmeans)
kmeans_df.head()

In [ ]:
# PCA Plot
plot_model(kmeans) 

In [ ]:
#Also you can plot Silhouette
plot_model(kmeans, plot='silhouette') 

In [ ]:
# Or you can plot Elbow etc.
plot_model(kmeans, plot='elbow') 

In [ ]:
# tunes the num_clusters model parameter using a predefined grid with the objective of optimizing a supervised learning metric as defined in the optimize param. 
tuned_kmeans = tune_model(model = 'kmeans', supervised_target = 'Survived')

In [ ]:
# Also, you can specify estimator for tuning.
tuned_kmeans = tune_model(model = 'kmeans', supervised_target = 'Survived', estimator='xgboost')

# *8. Some last words*

In one of my previous notebooks, I prepared a Simple Guide of PyCaret for how to use it for regression type of problems. After getting good feedbacks for PyCaret, I just want to prepare a new extended guide for PyCaret. My main objective was not improving the results. I just want to show as many as features and functions of PyCaret. You can find more detailed information from https://pycaret.org/

If you have any ideas to feedback please let me know in comments, and if you liked my work please don't forget to vote, thank you!

![](https://pycaret.org/wp-content/uploads/2020/03/Divi93_43.png)